<a href="https://colab.research.google.com/github/aswinsuresh-04/Digital_Image_and_Video_Processing-Lab/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import math
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import  DataLoader
import torch.nn.functional as f

device = 'cuda'



In [2]:
batch_size = 32

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])

trainset = datasets.MNIST(root = './root', train = True, download = True, transform = transform)
trainloader = DataLoader(trainset, batch_size = batch_size, shuffle = True)


100%|██████████| 9.91M/9.91M [00:01<00:00, 5.04MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 133kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.53MB/s]


In [3]:
class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(784, 2048),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(2048, 1024),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, 1),
        nn.Sigmoid(),
    )

  def forward(self, x):
    x = x.view(x.size(0), 784)
    output = self.model(x)



discriminator = Discriminator().to(device = device)


In [4]:
class Generator(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(
        nn.Linear(100, 256),
        nn.ReLU(),
        nn.Linear(256, 512),
        nn.ReLU(),
        nn.Linear(512, 1024),
        nn.ReLU(),
        nn.Linear(1024,784 ),
        nn.Tanh(),
    )
  def forward(self , x):
    output = self.model(x)
    output = output.view(x.size(0), 1, 28, 28)
    return output

generator = Generator().to(device = device)



In [5]:
lr = 0.0001
epoch = 50
loss_function = nn.BCELoss()

optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr = lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr = lr)

In [7]:
for epoch in range(epoch):
  for n, (samples, _) in enumerate(trainloader):
    samples = samples.to(device)
    sample_labels = torch.ones((batch_size, 1)).to(device)
    latent_space_samples = torch.randn((batch_size, 100)).to(device)
    generated_samples = generator(latent_space_samples)
    generated_samples_labels = torch.zeros((32, 1))
    labels = torch.ones((batch_size, 1))

    all_samples = torch.cat((samples, generated_samples))
    all_labels = torch.cat((labels, generated_samples_labels))

    #training the discriminator
    discriminator.zero_grad()
    output_discriminator = discriminator(all_samples)
    loss_discriminator = loss_function(input = output_discriminator, target = all_labels)
    loss_discriminator.backward()
    optimizer_discriminator.step()

    #data for training the generator
    latent_space_samples = torch.randn((batch_size, 100)).to(device)

    #training the generator
    generator.zero_grad()
    generated_samples = generator(latent_space_samples)
    output_discriminator = discriminator(generated_samples)
    loss_generator = loss_function(output_discriminator, labels)
    loss_generator.backward()
    optimizer_generator.step()


    if n == batch_size - 1:
      print(f'epoch : {epoch} decriminated loss : {loss_discriminator}')
      print(f'epoch : {epoch} generator loss : {loss_generator}')



In [9]:
for epoch in range(epoch):
    for n, (real_samples, _) in enumerate(trainloader):
        real_samples = real_samples.to(device)
        real_labels = torch.ones((real_samples.size(0), 1)).to(device)
        batch_size = real_samples.size(0)

        # Generate fake samples
        latent_space_samples = torch.randn((batch_size, 100)).to(device)
        fake_samples = generator(latent_space_samples)
        fake_labels = torch.zeros((batch_size, 1)).to(device)

        # Combine real and fake samples
        all_samples = torch.cat((real_samples, fake_samples))
        all_labels = torch.cat((real_labels, fake_labels))

        # Train the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = loss_function(output_discriminator, all_labels)
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Generate new latent space samples
        latent_space_samples = torch.randn((batch_size, 100)).to(device)

        # Train the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_on_generated = discriminator(generated_samples)
        loss_generator = loss_function(output_discriminator_on_generated, real_labels)  # Use real labels for generator training
        loss_generator.backward()
        optimizer_generator.step()

        if n % print_interval == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Batch [{n}/{len(trainloader)}]')
            print(f'Discriminator Loss: {loss_discriminator.item()}')
            print(f'Generator Loss: {loss_generator.item()}')
print('i am aswin')

i am aswin
